In [2]:
 # This is boilerplate code to correctly setup the settings to for notebook.
import os, sys
for root, dirs, files in os.walk(os.getcwd()):
    if root.split("/")[-1] == "pi0-analysis":
        pypath = os.getcwd() + "/pi0-analysis/analysis"

sys.path.insert(0, pypath)

from python.analysis.NotebookUtils import init_notebook
%init_notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/bhuller/Projects/xs_analysis/pi0-analysis/analysis
env: PYTHONPATH=/home/bhuller/Projects/xs_analysis/pi0-analysis/analysis


In [ ]:
from rich import print

from python.analysis import (
    Master, # Core library for Ntuple file manipulation
    BeamParticleSelection, # Defined selections for the beam particle.
    PFOSelection, # Defined selections for (Particle Flow Objects) PFOs
    Plots, # Plotting functions
    vector, # doing vector operations using Awkward.
    Tags, # Creating event tags
    cross_section, # Helper functions specifically for the cross section analysis code
    EventSelection # Helper functions for event selection
)

import awkward as ak # library for manipulating vectors faster than numpy
import numpy as np

cross_section.PlotStyler.SetPlotStyle(extend_colors = True, dpi = 100) # set the plot style to use (aesthetics)

ModuleNotFoundError: No module named 'python'

# File manipulation

In [1]:
# Open a nutple file (this can take a while depending on the file size/machine you are using)
mc = Master.Data("./../../../pi0-analysis/analysis/work/ROOTFiles/PDSPAnalyser/PDSPProd4a_MC_2GeV_sce_datadriven_ntuple_v09_81_00d01_set1.root", nTuple_type = Master.Ntuple_Type.PDSP)

NameError: name 'Master' is not defined